<a href="https://colab.research.google.com/github/epowell101/mscGNN-work/blob/main/raphtory_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install Raphtory

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 756.0/756.0 kB 60.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 124.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.0/40.0 MB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 341.8/341.8 kB 37.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 97.1 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 9.0.0
    Uninstalling pyarrow-9.0.0:
      Successfully uninstalled pyarrow-9.0.0
  Attempting uninstall: pandas
    Found existing installation: pandas 1.5.3
    Uninstalling pandas-1.5.3:
      Successfully uninstalled pandas-1.5.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas

In [16]:
print("****Downloading Data****")
!curl -o /tmp/lotr.csv https://raw.githubusercontent.com/Raphtory/Data/main/lotr.csv
!curl -o /tmp/lotr_properties.csv https://raw.githubusercontent.com/Raphtory/Data/main/lotr_properties.csv
!curl -o /tmp/lotr.db https://raw.githubusercontent.com/Raphtory/Data/main/lotr.db
print("****LOTR GRAPH STRUCTURE****")
!head -n 3 /tmp/lotr.csv
print("****LOTR GRAPH PROPERTIES****")
!head -n 3 /tmp/lotr_properties.csv

****Downloading Data****
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 52206  100 52206    0     0   375k      0 --:--:-- --:--:-- --:--:--  377k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   686  100   686    0     0   6158      0 --:--:-- --:--:-- --:--:--  6180
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 69632  100 69632    0     0   402k      0 --:--:-- --:--:-- --:--:--  404k
****LOTR GRAPH STRUCTURE****
Gandalf,Elrond,33
Frodo,Bilbo,114
Blanco,Marcho,146
****LOTR GRAPH PROPERTIES****
Aragorn,men,male
Gandalf,ainur,male
Goldberry,ainur,female


In [22]:
import csv
import pandas as pd
from raphtory import Graph

structure_file = "/tmp/lotr.csv"
properties_file = "/tmp/lotr_properties.csv"
graph = Graph()  # when a 1 or 0 is placed here per https://docs.raphtory.com/en/v0.5.2/getting_started/intro_tutorials/lotr.html an error
# TypeError: Graph.__new__() takes 0 positional arguments but 1 was given

In [24]:
with open(structure_file, 'r') as csvfile:
    datareader = csv.reader(csvfile)
    for row in datareader:

        source_node = row[0]
        destination_node = row[1]
        timestamp = int(row[2])

        graph.add_vertex(timestamp, source_node, {"vertex_type": "Character"})
        graph.add_vertex(timestamp, destination_node, {"vertex_type": "Character"})
        graph.add_edge(timestamp, source_node, destination_node, {"edge_type": "Character_Co-occurence"})

In [25]:
print("Earliest time: %i" % graph.earliest_time())
print("Latest time: %i" % graph.latest_time())

print("Number of vertices: %i" % len(graph.vertices()))
print("Number of edges: %i" % len(graph.edges()))

Earliest time: 33
Latest time: 32674
Number of vertices: 139
Number of edges: 701


In [26]:
print("Gandalf's degree at 1000: %i" % graph.at(1000).vertex("Gandalf").degree())

print("Gandalf's degree at 10,000: %i" % graph.vertex("Gandalf").at(10000).degree())

Gandalf's degree at 1000: 4
Gandalf's degree at 10,000: 26


In [34]:
with open(properties_file, 'r') as csvfile:
    datareader = csv.reader(csvfile)
    for row in datareader:
        graph.add_vertex_properties(row[0], {"race": row[1],"gender": row[2]})

In [40]:
result = []
vertices = list(graph.vertices())

for vertex in vertices:
    race_property = vertex.properties.get("race") # Use get method to avoid KeyError which was the result of the example in tutorials
    if race_property == "dwarf": # Check if the race property is "dwarf"
        interactions = sum([len(e.explode()) for e in vertex.edges()])
        latest = vertex.latest_time()
        result.append({"timestamp": latest, "name": vertex.name(), "interactions": interactions })

pd.DataFrame(result).sort_values(by="interactions", ascending=False)



,timestamp,name,interactions
3,31247,Gimli,185
1,31129,Glóin,31
2,10938,Balin,14
0,9605,Thorin,5
